In [ ]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import ast

path = 'mem/data.parquet'

table = pq.read_table(path)
df_parquet = table.to_pandas()

columns_array = ['mbMap', 'mbScrewMap' ,'psuMap', 'psuScrewMap', 'ursaMap', 'ursaScrewMap', 'wlanMap', 'wlanScrewMap', 'cable_map', 'cable_trns_map',
                  'cable_single_shield_map', 'cable_double_shield_map', 'cable_circle_shield_map', 'emi_tape_map', 'ferrit_map', 'vData',
                  'hData', 'freqMarginH', 'freqMarginV', 'freqCriticH', 'freqCriticV', 'freqPotentialH', 'freqPotentialV']

columns_float = ['remBiasH', 'remBiasV']
columns_int = ['inch', 'ranking']

data_dict = {}

# str to list in columns_array
for column_name in columns_array:
    df_parquet[column_name] = df_parquet[column_name].apply(ast.literal_eval)
    print("column name:",column_name)

# str to float in columns_float
for column_name in columns_float:
    df_parquet[column_name] = df_parquet[column_name].astype('float32')#'float32'
    print("column name:",column_name)

# str to int in columns_int
for column_name in columns_int:
    df_parquet[column_name] = df_parquet[column_name].astype('uint16')#'uint16'
    print("column name:",column_name)

In [ ]:
#filter unnecessary labels.
columns = ['inch', 'mbCode', 'mbMap', 'pnlCode', 'cell_code',
       'panel_vendor', 'mbScrewMap','psuCode', 'psuMap',
       'psuScrewMap', 'ursaCode','ursaMap',
       'ursaScrewMap', 'wlanCode', 'wlanMap', 'wlanScrewMap',
       'cable_map', 'cable_trns_map', 'cable_single_shield_map',
       'cable_double_shield_map', 'cable_circle_shield_map',
        'emi_tape_map', 'ferrit_map', 'test_name', 'vData', 'hData', 'remBiasH', 'remBiasV', 'freqMarginH',
       'freqMarginV', 'freqCriticH', 'freqCriticV', 'freqPotentialH',
       'freqPotentialV', 'ranking'] 
df_parquet_cropped = df_parquet[columns]

dict_result = {}

for column_name in df_parquet_cropped.columns:
    column_shape = np.asarray(df_parquet_cropped[column_name][0]).shape

    if column_shape == ():
        dict_result[column_name] = df_parquet_cropped[column_name].tolist()
        print("key:", column_name, "shape:", column_shape)
    elif column_shape == (160, 90):
        dict_result[column_name] = df_parquet_cropped[column_name].apply(lambda x: [elem for sublist1d in x for elem in sublist1d]).tolist()
        print("key:", column_name, "shape:", column_shape)
    elif column_shape in [(64, 160, 90), (96, 160, 90)]:
        dict_result[column_name] = df_parquet_cropped[column_name].apply(lambda x: [elem for sublist2d in x for sublist1d in sublist2d for elem in sublist1d]).tolist()
        print("key:", column_name, "shape:", column_shape)
    elif column_shape == (971,):
        dict_result[column_name] = df_parquet_cropped[column_name].tolist()
        print("key:", column_name, "shape:", column_shape)

df_parquet_cropped_last = pd.DataFrame(dict_result)
print("islem bitti")
df_parquet_cropped_last


In [ ]:
map_columns = ['mbMap','mbScrewMap','psuMap','psuScrewMap','ursaMap','ursaScrewMap',
                'wlanMap','wlanScrewMap','cable_map','cable_trns_map', 'cable_single_shield_map',
                'cable_double_shield_map','cable_circle_shield_map', 'emi_tape_map', 'ferrit_map',
                'freqMarginH', 'freqMarginV','freqCriticH', 'freqCriticV','freqPotentialH', 'freqPotentialV']
# Mapping şeklinde olan columnların array şeklinde kalması model oluşturmama engel oluyordu ben de bu arrayleri şu formatta böldüm:
# Mesela bir arrayde 971 adet element varsa, o arrayin ismi de örek olarak vData olsun, vData_1,vData_2,...,vData_970,vData_971 şeklinde 971 tane column yarattım. Siz farklı bir düşünürseniz burayla oynayabilirsiniz.

float_columns = ['vData','hData']
df_last = df_parquet_cropped_last
for col in map_columns:
  df_last = df_last.join(pd.DataFrame(df_last[col].values.tolist()).add_prefix(col+"_").astype('int8'))
  df_last = df_last.drop(columns=col)
for col in float_columns:
  df_last = df_last.join(pd.DataFrame(df_last[col].values.tolist()).add_prefix(col+"_"))
  df_last = df_last.drop(columns=col)
df_last

In [ ]:
cat_columns = ['inch', 'mbCode', 'pnlCode', 'cell_code', 'panel_vendor', 'psuCode', 'ursaCode', 'wlanCode', 'test_name']

for cc in cat_columns:
    df_last[cc] = pd.Categorical(df_last[cc])
    df_last[cc] = df_last[cc].cat.codes

df_last[cat_columns]

In [ ]:
df_last

In [ ]:
df_last.to_pickle('data_150.pkl') # 150 veri adedi